In [2]:
import pandas as pd
import re
from Bio import SeqIO
import numpy as np


In [6]:
hcov= 'SARS2' #choose name

df=pd.DataFrame()
with open('genomic_analyses//SEQ//MSA//final_seq_'+str(hcov)+'_aligned.fasta') as file:
    for seq_rec in SeqIO.parse(file, 'fasta'):
        seq_id= seq_rec.description
        if seq_id:
            record= {'id': seq_id, 'seq': str(seq_rec.seq.lower())}
            df=df.append(record, ignore_index=True)
print(len(df), df)

1493                                          id  \
0                               NC_045512.2   
1               hCoV-19/Namibia/N25837/2021   
2           hCoV-19/Guinea/LFHVG-G0623/2021   
3         hCoV-19/Kenya/ILRI_COVM02475/2021   
4     hCoV-19/Palestine/QIBPS-OM_12_S9/2022   
...                                     ...   
1488   hCoV-19/Slovakia/BA_22_00029745/2022   
1489    hCoV-19/Spain/MD-HGUGM-6507189/2022   
1490            hCoV-19/Singapore/4932/2022   
1491            hCoV-19/Singapore/4968/2022   
1492      hCoV-19/Canada/MB-CPL-421038/2022   

                                                    seq  
0     atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...  
1     atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...  
2     atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...  
3     atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...  
4     atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...  
...                                                 ...  
1488  atgtttgtttttcttgtt

In [126]:
# print(list(df['id']))

In [7]:
listresult=[]
#look in id for the year of collection
for i, row in df.iterrows():

    resultre=re.search(r'(?<=\/)((20[0-9]{2})|(19[0-9]{2}))$', row['id'])
    if resultre:
        result=resultre.group()
    else:
        result=np.nan


    
    listresult.append(result)
        
df['year']=listresult
print(len(df))
df=df[df['year'].notnull()]
print(len(df))

1493
1492


In [8]:
df1=pd.DataFrame()
# for each unique year, sample 10 sequences randomly to eventually make the tree with
for year in df.year.unique():
    count=len(df[df.year == year])
    print(year, count)
    if count < 50:
        # print(df[df.year == year].sample(n=count,replace=False))
        dfnew=df[df.year == year].sample(n=count,replace=False)

    else:
        # print(df[df.year == year].sample(n=10,replace=False))
        dfnew=df[df.year == year].sample(n=50,replace=False)
    df1=df1.append(dfnew, ignore_index=True)

2021 648
2022 657
2020 186
2019 1


In [9]:
df1['id']=df1['id'].str.replace(' ', '')
df1

,id,seq,year
0,hCoV-19/France/MC-CERBAHC-1240226/2021,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...,2021
1,hCoV-19/Croatia/HRV000_126/2021,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...,2021
2,hCoV-19/Mongolia/10011941/2021,atgtttgtttttcttgttttattgccactagtttctagtcagtgtg...,2021
3,hCoV-19/Mexico/NLE_INER_IMSS_3176/2021,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...,2021
4,hCoV-19/ElSalvador/INC-LNSP-097/2021,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...,2021
...,...,...,...
146,hCoV-19/France/PAC-IHU-1128/2020,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...,2020
147,hCoV-19/Haiti/33984/2020,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...,2020
148,hCoV-19/Mexico/GTO-InDRE_115/2020,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...,2020
149,hCoV-19/Luxembourg/LNS8602041/2020,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...,2020


In [10]:
def dataframetofasta(dataframe, fastaname):
        #create fasta file
        file= fastaname + ".fasta"
        f= open(file, 'a')
        #write to the file
        for i, row in dataframe.iterrows():
            lines='>'+ row['id']+'\n'+ row['seq'] + "\n"
            f.write(lines)
        f.close()


#change name of the file with every run
dataframetofasta(df1, 'genomic_analyses//SEQ//MSA//'+str('final_seq_downsampled_'+str(hcov)+ '_MSA'))

